In [10]:
import pandas as pd
import numpy as np
from haversine import haversine

,num, time, latitude, longitude, sog, cog, hdg, anchor_drag, time_gap, d_km/s, dd_km/s

In [11]:
ulsan_test = pd.read_csv("../기상청공모전/data/ulsan_anch_drag_test.csv")
ulsan_test = ulsan_test[['ulsan_anch_drag_test.time','ulsan_anch_drag_test.num', 'ulsan_anch_drag_test.lat','ulsan_anch_drag_test.lon', 'ulsan_anch_drag_test.sog','ulsan_anch_drag_test.cog', 'ulsan_anch_drag_test.hdg']]
ulsan_test.columns = ['time', 'num' ,'latitude','longitude','sog','cog','hdg']
ulsan_test = ulsan_test[['num','time','latitude','longitude','sog','cog','hdg']]
ulsan_test = ulsan_test.replace(regex=r'["]', value='')
ulsan_test = ulsan_test.replace(regex=r'[N]', value='')
ulsan_test = ulsan_test.replace(regex=r'[E]', value='')
ulsan_test['time'] = pd.to_datetime(ulsan_test['time'])
ulsan_test['latitude'] = ulsan_test['latitude'].astype(float)
ulsan_test['longitude'] = ulsan_test['longitude'].astype(float)
ulsan_test['time_gap'] = 0
ulsan_test['d_km/s'] = 0
ulsan_test['dd_km/s'] = 0
ulsan_test


,num,time,latitude,longitude,sog,cog,hdg,time_gap,d_km/s,dd_km/s
0,1001,2022-08-24 14:41:34,35.442767,129.415550,0.6,231.0,116,0,0,0
1,1001,2022-08-24 14:41:44,35.442767,129.415500,0.8,240.0,115,0,0,0
2,1001,2022-08-24 14:41:53,35.442750,129.415467,0.7,258.0,116,0,0,0
3,1001,2022-08-24 14:42:04,35.442750,129.415417,0.8,254.0,116,0,0,0
4,1001,2022-08-24 14:42:14,35.442733,129.415367,1.0,253.0,117,0,0,0
...,...,...,...,...,...,...,...,...,...,...
186508,1117,2023-01-17 19:44:27,35.456302,129.424155,0.1,325.8,312,0,0,0
186509,1117,2023-01-17 19:50:27,35.456108,129.424060,0.3,71.3,338,0,0,0
186510,1117,2023-01-17 19:56:27,35.456153,129.424177,0.1,195.9,325,0,0,0
186511,1117,2023-01-17 19:59:24,35.456200,129.424007,0.1,274.4,324,0,0,0


In [12]:
for i in range(1, len(ulsan_test)):
    if ulsan_test.loc[i, 'time_gap'] == 0:
        dt = (pd.to_datetime(ulsan_test.loc[i, 'time']) - pd.to_datetime(ulsan_test.loc[i-1, 'time'])).total_seconds()
        if dt != 0:
            ulsan_test.loc[i, 'time_gap'] = dt
        else:
            a = ulsan_test.loc[(ulsan_test['num'] == ulsan_test.loc[i, 'num']) & (ulsan_test['time'] == ulsan_test.loc[i, 'time'])].index.tolist()
            ulsan_test.loc[a, 'time_gap'] = 1 / len(a)

# 결과 출력
print(ulsan_test)


         num                time   latitude   longitude  sog    cog  hdg  \
0       1001 2022-08-24 14:41:34  35.442767  129.415550  0.6  231.0  116   
1       1001 2022-08-24 14:41:44  35.442767  129.415500  0.8  240.0  115   
2       1001 2022-08-24 14:41:53  35.442750  129.415467  0.7  258.0  116   
3       1001 2022-08-24 14:42:04  35.442750  129.415417  0.8  254.0  116   
4       1001 2022-08-24 14:42:14  35.442733  129.415367  1.0  253.0  117   
...      ...                 ...        ...         ...  ...    ...  ...   
186508  1117 2023-01-17 19:44:27  35.456302  129.424155  0.1  325.8  312   
186509  1117 2023-01-17 19:50:27  35.456108  129.424060  0.3   71.3  338   
186510  1117 2023-01-17 19:56:27  35.456153  129.424177  0.1  195.9  325   
186511  1117 2023-01-17 19:59:24  35.456200  129.424007  0.1  274.4  324   
186512  1117 2023-01-17 20:05:24  35.456177  129.424080  0.3  150.6  326   

        time_gap  d_km/s  dd_km/s  
0            0.0       0        0  
1           10.

In [13]:
for i in range(1, len(ulsan_test)):
    ulsan_test.loc[i, 'd_km/s'] = haversine((ulsan_test.loc[i, 'latitude'],ulsan_test.loc[i, 'longitude']),(ulsan_test.loc[i-1, 'latitude'],ulsan_test.loc[i-1, 'longitude']),unit = 'km') / ulsan_test.loc[i, 'time_gap']

In [14]:
for i in range(2, len(ulsan_test)):
    ulsan_test.loc[i, 'dd_km/s_2'] = (ulsan_test.loc[i, 'd_km/s'] - ulsan_test.loc[i-1, 'd_km/s']) / ulsan_test.loc[i,'time_gap']

In [ ]:
# 각 배 번호의 1번째 2번째 삭제
ship_num_busan = ulsan_test['num'].unique()

_drop_busan = []
for i in ship_num_busan:
    k = ulsan_test[ulsan_test["num"] == i].index[0]
    _drop_busan.append(k)
    k += 1
    _drop_busan.append(k)
    
ulsan_test = ulsan_test[~ulsan_test.index.isin(_drop_busan)].reset_index(drop=True)

In [ ]:
ulsan_test

,num,time,latitude,longitude,sog,cog,hdg,time_gap,d_km/s,dd_km/s,dd_km/s_2
0,1001,2022-08-24 14:41:34,35.442767,129.415550,0.6,231.0,116,0.0,0.000000,0,NaN
1,1001,2022-08-24 14:41:44,35.442767,129.415500,0.8,240.0,115,10.0,0.000453,0,NaN
2,1001,2022-08-24 14:41:53,35.442750,129.415467,0.7,258.0,116,9.0,0.000393,0,-6.661376e-06
3,1001,2022-08-24 14:42:04,35.442750,129.415417,0.8,254.0,116,11.0,0.000412,0,1.706832e-06
4,1001,2022-08-24 14:42:14,35.442733,129.415367,1.0,253.0,117,10.0,0.000491,0,7.903957e-06
...,...,...,...,...,...,...,...,...,...,...,...
186508,1117,2023-01-17 19:44:27,35.456302,129.424155,0.1,325.8,312,178.0,0.000078,0,3.245203e-07
186509,1117,2023-01-17 19:50:27,35.456108,129.424060,0.3,71.3,338,360.0,0.000065,0,-3.772269e-08
186510,1117,2023-01-17 19:56:27,35.456153,129.424177,0.1,195.9,325,360.0,0.000033,0,-8.877639e-08
186511,1117,2023-01-17 19:59:24,35.456200,129.424007,0.1,274.4,324,177.0,0.000092,0,3.351061e-07


In [ ]:
ulsan_test.to_csv('../기상청공모전/data/ulsan_test(der).csv')